In [1]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

# Hàm xác định loại album
def determine_album_type(track_count):
    if track_count <= 3:
        return "Single"
    elif 4 <= track_count <= 6:
        return "EP"
    else:
        return "Regular"

# Hàm lấy tất cả bài hát của nghệ sĩ
def get_artist_tracks_all(artist_name):
    client_id = "c7e1fe3ffe674920a01f9b016e6ae5df"
    client_secret = "215c6808dea74656b3629b306182ac4b"
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

    # 🔎 Tìm nghệ sĩ theo tên
    results = sp.search(q=artist_name, type='artist', limit=1)
    if not results['artists']['items']:
        print("Không tìm thấy nghệ sĩ.")
        return pd.DataFrame()

    artist_id = results['artists']['items'][0]['id']

    track_data = []
    seen_tracks = set()
    album_tracks_count = {}


    offset = 0
    while True:
        albums = sp.artist_albums(artist_id, album_type='album,single', limit=50, offset=offset)
        if not albums['items']:
            break
        for album in albums['items']:
            album_id = album['id']
            album_name = album['name']
            album_release_date = album.get('release_date', 'Unknown')
            album_owner = album['artists'][0]['name']

            album_info = sp.album(album_id)
            label = album_info.get('label', 'Unknown')

           
            tracks = sp.album_tracks(album_id)['items']
            album_tracks_count[album_name] = len(tracks)  # Ghi lại số lượng bài hát

            for track in tracks:
                track_id = track['id']
                if track_id in seen_tracks:
                    continue
                seen_tracks.add(track_id)

                track_title = track['name']
                link_spotify = track['external_urls']['spotify']
                featured_artists = [artist['name'] for artist in track['artists'] if artist['id'] != artist_id]
                featured_artists = ", ".join(featured_artists) if featured_artists else "None"

                # Xử lý ngày phát hành
                try:
                    album_release_date = pd.to_datetime(album_release_date, errors='coerce').strftime("%d/%m/%Y")
                except:
                    album_release_date = "Unknown"

                track_data.append([album_name, track_title, album_release_date, featured_artists, album_owner, label, link_spotify])
        offset += 50  

   
    top_tracks = sp.artist_top_tracks(artist_id, country="US")['tracks']
    for track in top_tracks:
        track_id = track['id']
        if track_id in seen_tracks:
            continue
        seen_tracks.add(track_id)

        track_title = track['name']
        link_spotify = track['external_urls']['spotify']
        album = track['album']
        album_name = album['name']
        album_release_date = album.get('release_date', 'Unknown')
        album_owner = album['artists'][0]['name']
        featured_artists = [artist['name'] for artist in track['artists'] if artist['id'] != artist_id]
        featured_artists = ", ".join(featured_artists) if featured_artists else "None"

        # 🔹 Lấy thông tin label của album
        album_info = sp.album(album['id'])
        label = album_info.get('label', 'Unknown')

        try:
            album_release_date = pd.to_datetime(album_release_date, errors='coerce').strftime("%d/%m/%Y")
        except:
            album_release_date = "Unknown"

        track_data.append([album_name, track_title, album_release_date, featured_artists, album_owner, label, link_spotify])
        album_tracks_count[album_name] = album_tracks_count.get(album_name, 0) + 1  # Cập nhật số bài hát


    columns = ["album_name", "tracklist", "release_date", "featured_artists", "album_owner_", "provided_by", "Link_Spotify"]
    df = pd.DataFrame(track_data, columns=columns)

    df['album_type'] = df['album_name'].map(lambda x: determine_album_type(album_tracks_count.get(x, 0)))
    df = df.sort_values(by=["album_name", "tracklist"], ascending=[True, True])
 

    return df

# Chạy chương trình
if __name__ == "__main__":
    artist_name_Spotify = 'Ngắn'
    df_tracks = get_artist_tracks_all(artist_name_Spotify)




C:\Users\user\AppData\Local\Temp\ipykernel_20452\1693337626.py:64: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  album_release_date = pd.to_datetime(album_release_date, errors='coerce').strftime("%d/%m/%Y")


In [2]:

df_tracks.to_excel(f"{artist_name_Spotify}_songSpotify.xlsx", index=False)
df_tracks.head()

,album_name,tracklist,release_date,featured_artists,album_owner_,provided_by,Link_Spotify,album_type
26,120Bpm (ft. Long Mộng Gà),120Bpm (ft. Long Mộng Gà),10/03/2023,Antoneus Maximus,Ngắn,MaiDao Music,https://open.spotify.com/track/3QSw9fOs1Qe14tg...,Single
64,9X,9X,04/05/2020,"DLBlack, Mêk Team",DLBlack,Zing MP3,https://open.spotify.com/track/2Gj8BVWfiqHTykY...,Single
52,Bình Dân,Bình Dân,03/09/2021,None,Ngắn,Yin Yang Media,https://open.spotify.com/track/3dhDxLqZScUdYDx...,Single
53,Bình Dân,Bình Dân - Beat,09/03/2021,None,Ngắn,Yin Yang Media,https://open.spotify.com/track/0eof9V6sNRlPtiQ...,Single
32,Bụi Thiên Thần,Cuộc Đời Này Ngắn,27/07/2022,None,Ngắn,LOOPS Music,https://open.spotify.com/track/3UmZvOmW9QM7W1F...,EP
